In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def sigmoid(x):
    return (1/(1+np.exp(-x)));

def get_weights(cur_layer,next_layer):
    """ Returns random weights to start with. including last row (all ones) for bias
    where cur_layer is no of neurons for current layer and
    next_layer is no of neurons for next layer"""
    z=np.random.random((cur_layer-1,next_layer));
    b=np.ones((1,next_layer));
    w=np.vstack((z,b));
    return w;

def perceptron(input_x,weights):
    return sigmoid(np.dot(input_x,weights));

    

In [3]:
inp_x=np.linspace(0,7,8);
act_m = 4;
y=4*inp_x;

In [4]:
print(inp_x)

[ 0.  1.  2.  3.  4.  5.  6.  7.]


In [11]:
lr=0.1;
guess_m= -1;
num_x=len(inp_x);
num_of_iter = 100;

for i in np.arange(num_of_iter ):
    dCdM = (sum(abs(guess_m*inp_x - y)*guess_m));
    guess_m=guess_m - (lr/num_x)*dCdM;
    print("dCdM is %f new_guess m is %f "%(dCdM,guess_m))

dCdM is -140.000000 new_guess m is 0.750000 
dCdM is 68.250000 new_guess m is -0.103125 
dCdM is -11.847773 new_guess m is 0.044972 
dCdM is 4.980253 new_guess m is -0.017281 
dCdM is -1.943833 new_guess m is 0.007017 
dCdM is 0.784516 new_guess m is -0.002790 
dCdM is -0.312646 new_guess m is 0.001119 
dCdM is 0.125241 new_guess m is -0.000447 
dCdM is -0.050067 new_guess m is 0.000179 
dCdM is 0.020032 new_guess m is -0.000072 
dCdM is -0.008012 new_guess m is 0.000029 
dCdM is 0.003205 new_guess m is -0.000011 
dCdM is -0.001282 new_guess m is 0.000005 
dCdM is 0.000513 new_guess m is -0.000002 
dCdM is -0.000205 new_guess m is 0.000001 
dCdM is 0.000082 new_guess m is -0.000000 
dCdM is -0.000033 new_guess m is 0.000000 
dCdM is 0.000013 new_guess m is -0.000000 
dCdM is -0.000005 new_guess m is 0.000000 
dCdM is 0.000002 new_guess m is -0.000000 
dCdM is -0.000001 new_guess m is 0.000000 
dCdM is 0.000000 new_guess m is -0.000000 
dCdM is -0.000000 new_guess m is 0.000000 
dCdM is